In [1]:
!pip3 install torch torchvision torchaudio
!pip3 install transformers
!pip3 install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Using cached scikit_learn-1.3.2-cp39-cp39-macosx_12_0_arm64.whl (9.5 MB)
  Using cached scipy-1.11.4-cp39-cp39-macosx_12_0_arm64.whl (29.7 MB)
  Using cached sentencepiece-0.1.99-cp39-cp39-macosx_11_0_arm64.whl (1.2 MB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d62c467c27bdb1de0cae4427632b6703542c

In [23]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from tqdm import tqdm
import torch
import json
import requests

In [24]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [25]:
with open("/Users/arush/downloads/IR_LAMP/Personalized_News_Categorization_validation_input.json", "r") as f:
    json_data = json.load(f)

In [26]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
golds = []
for user in tqdm(json_data):
    predictions = {}
    query = user['input']
    inputs = tokenizer(query, return_tensors = "pt")
    outputs = llm_model.generate(**inputs)
    
    predictions["output"] = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0]
    predictions["id"] = user["id"]
    golds.append(predictions)
final_json = {}
final_json["task"] = "LaMP_2"
final_json["golds"] = golds

100%|█████████████████████████████████████████| 200/200 [25:09<00:00,  7.55s/it]


In [28]:
with open("pred_news_categorization_no_personalization.json", "w") as f:
    json.dump(final_json, f)